# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore') 



# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,29.21,75,43,5.34,TK,2024-06-10
1,1,st. john's,47.5649,-52.7093,11.66,95,20,8.23,CA,2024-06-10
2,2,west island,-12.1568,96.8225,24.99,94,75,6.69,CC,2024-06-10
3,3,port-aux-francais,-49.3500,70.2167,2.87,70,58,14.12,TF,2024-06-10
4,4,grytviken,-54.2811,-36.5092,-4.30,85,56,5.19,GS,2024-06-10


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_narrow_lats_df = city_data_df[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30)]

# Drop any rows with null values
ideal_weather_city_df = cities_narrow_lats_df.dropna()

# Display sample data
ideal_weather_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atafu village,-8.5421,-172.5159,29.21,75,43,5.34,TK,2024-06-10
2,2,west island,-12.1568,96.8225,24.99,94,75,6.69,CC,2024-06-10
6,6,lihue,21.9789,-159.3672,29.58,73,40,6.69,US,2024-06-10
12,12,adamstown,-25.0660,-130.1015,22.68,79,100,6.69,PN,2024-06-10
14,14,khandyga,62.6667,135.6000,22.53,29,45,0.81,RU,2024-06-10
...,...,...,...,...,...,...,...,...,...,...
563,563,suntar,62.1444,117.6319,26.45,26,11,2.32,RU,2024-06-10
564,564,fortaleza,-3.7227,-38.5247,26.23,89,20,2.57,BR,2024-06-10
565,565,karratha,-20.7377,116.8463,26.08,32,0,5.91,AU,2024-06-10
566,566,ilam,26.9167,87.9167,29.30,60,74,2.54,NP,2024-06-10


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city_df[['City', 'Country','Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atafu village,TK,-8.5421,-172.5159,75,
2,west island,CC,-12.1568,96.8225,94,
6,lihue,US,21.9789,-159.3672,73,
12,adamstown,PN,-25.0660,-130.1015,79,
14,khandyga,RU,62.6667,135.6000,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
        
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
atafu village - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
lihue - nearest hotel: Kauai Palms
adamstown - nearest hotel: No hotel found
khandyga - nearest hotel: Апельсин
saint-pierre - nearest hotel: Tropic Hotel
mandiana - nearest hotel: Hôtel Prince
mirnyy - nearest hotel: Зарница
vila do maio - nearest hotel: Residencial Jardins De Maio
broken hill - nearest hotel: Ibis Styles
sorong - nearest hotel: Hotel Membramo Marina
aldan - nearest hotel: гостинница Велком Плаза
port-vila - nearest hotel: Holiday Inn Resort Vanuatu
mata-utu - nearest hotel: Gîte Oceania
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
mejit - nearest hotel: No hotel found
virginia - nearest hotel: Quirk Hotel
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
uturoa - nearest hotel: Hawaiki Nui hotel
itoman - nearest hotel: No hotel found
fuerte olimpo - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atafu village,TK,-8.5421,-172.5159,75,No hotel found
2,west island,CC,-12.1568,96.8225,94,Cocos Village Bungalows
6,lihue,US,21.9789,-159.3672,73,Kauai Palms
12,adamstown,PN,-25.0660,-130.1015,79,No hotel found
14,khandyga,RU,62.6667,135.6000,29,Апельсин
...,...,...,...,...,...,...
563,suntar,RU,62.1444,117.6319,26,No hotel found
564,fortaleza,BR,-3.7227,-38.5247,89,Hotel Catedral
565,karratha,AU,-20.7377,116.8463,32,Karratha International Hotel
566,ilam,NP,26.9167,87.9167,60,Hotel Pokhereli


In [7]:
# Export the Hotel into a csv
hotel_df.to_csv("output_data/hotel.csv", index_label="City_ID")

In [8]:
# Read saved data
hotel_df = pd.read_csv("output_data/hotel.csv", index_col="City_ID")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,atafu village,TK,-8.5421,-172.5159,75,No hotel found
2,west island,CC,-12.1568,96.8225,94,Cocos Village Bungalows
6,lihue,US,21.9789,-159.3672,73,Kauai Palms
12,adamstown,PN,-25.0660,-130.1015,79,No hotel found
14,khandyga,RU,62.6667,135.6000,29,Апельсин


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)